In [ ]:
'''

******Analytical Exercise
 
Banc ABC is a large US Bank that is protected by F5, Inc. They have approached F5 with a special request. They have set up special infrastructure in their environment to service financial aggregators like Mint and 
Plaid who login in to accounts on behalf of Banc ABC’s customers. Traditionally no attention has been paid to these aggregators and they have been allowed unfettered access to Banc ABC’s systems.

A new CISO has taken the reins at Banc ABC and has requested that we look at all the aggregator transactions and provide her with a report. She is specifically concerned about the rate at which aggregators are 
logging into accounts. She is proposing limiting each aggregator to 1 login per account per 10 min interval and would like to know what impact this will have on the aggregators.


Data Provided
You have been provided with two CSV files for the purpose of this exercise.

1.	Aggregator IP List.csv
Contains a list of all the allowlisted (whitelisted) IPs that belong to known financial aggregators that are allowed by Banc ABC to access their customer’s accounts. The format of the file is
IP: contains the IP addresses belonging to the aggregators that have been allowlisted
Aggregator: contains the name of the aggregator that owns the respective allowlisted IP address

2.	Login Transactions.csv
This file contains all the logins observed by F5 over a 24 hour period on Banc ABC’s special Aggregator ONLY endpoint. This endpoint was set up specifically to process transactions for the allowlisted aggregators so as not 
to interfere with regular customer traffic which uses a different endpoint.

The format of the file is:
IP: This is the IP address from which the login transaction came
LoginSuccess: This is an indicator of whether or not the login was successful i.e. had the correct accountName and password pair
AccountName: this is the account that the transaction was trying to log into
Timestamp: this is the epoch timestamp at which the respective login transaction was received
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Required

1.	Which aggregators are accessing Banc ABC’s systems?
2.	How much volume are the aggregators sending?
3.	How many individual user accounts are being accessed by the aggregators?
4.	What is the login success rate of these aggregators and is this in line with what would be expected in your opinion?
5.	What is the average number of transactions each aggregator sends per 10min interval?
6.	What is the maximum number of transactions each aggregator sends per 10min interval?
7.	What would be the impact of the CISO’s proposal to limit each aggregator to 1 login per account per 10 min interval?

Guidance
●	Feel free to use any analytical tools you feel comfortable with to perform the analysis and prepare the presentation for the CISO. We recommend, but do not require, use of Jupyter notebooks.
●	Be prepared to show your code or worksheets and to answer questions about your thought and analytical processes.
●	Presentation will be over Zoom to the Banc ABC CISO and will be scheduled for 1 hour, including questions.

You have 3 days to prepare the deck and email it to the Banc ABC CISO (please email directly to the Threat Analytics & Reporting team distro: tar@f5.com)


'''

In [ ]:
import pandas as pd
from datetime import datetime, timedelta

# Cargar los archivos CSV
aggregator_ip_list = pd.read_csv('Aggregator IP List.csv')
login_transactions = pd.read_csv('Login Transactions.csv')

# Convertir los timestamps de epoch a formato datetime
login_transactions['Timestamp'] = pd.to_datetime(login_transactions['Timestamp'], unit='s')

# Merge de login transactions con la lista de IPs permitidas para identificar los agregadores
merged_data = pd.merge(login_transactions, aggregator_ip_list, on='IP', how='inner')

# Responder a las preguntas

# 1. ¿Qué agregadores están accediendo a los sistemas de Banc ABC?
aggregators_accessing_systems = merged_data['Aggregator'].unique()
print("Aggregators accessing Banc ABC’s systems:", aggregators_accessing_systems)

# 2. ¿Qué volumen están enviando los agregadores?
volume_per_aggregator = merged_data['Aggregator'].value_counts()
print("Volume per aggregator:\n", volume_per_aggregator)

# 3. ¿Cuántas cuentas de usuario individuales están siendo accedidas por los agregadores?
accounts_accessed_per_aggregator = merged_data.groupby('Aggregator')['AccountName'].nunique()
print("Number of individual user accounts accessed by aggregators:\n", accounts_accessed_per_aggregator)

# 4. ¿Cuál es la tasa de éxito de inicio de sesión de estos agregadores y está en línea con lo esperado?
success_rate_per_aggregator = merged_data.groupby('Aggregator')['LoginSuccess'].mean()
print("Login success rate per aggregator:\n", success_rate_per_aggregator)

# 5. ¿Cuál es el número promedio de transacciones que cada agregador envía por intervalo de 10 minutos?
merged_data.set_index('Timestamp', inplace=True)
average_transactions_per_10min = merged_data.groupby('Aggregator').resample('10T').size().groupby('Aggregator').mean()
print("Average number of transactions per 10-minute interval per aggregator:\n", average_transactions_per_10min)

# 6. ¿Cuál es el número máximo de transacciones que cada agregador envía por intervalo de 10 minutos?
max_transactions_per_10min = merged_data.groupby('Aggregator').resample('10T').size().groupby('Aggregator').max()
print("Maximum number of transactions per 10-minute interval per aggregator:\n", max_transactions_per_10min)

# 7. ¿Cuál sería el impacto de la propuesta del CISO de limitar cada agregador a 1 inicio de sesión por cuenta por intervalo de 10 minutos?
def limit_logins(df):
    df = df.sort_values(by='Timestamp')
    df['Allowed'] = df.groupby(['AccountName', 'Aggregator'])['Timestamp'].diff().gt(pd.Timedelta(minutes=10))
    df['Allowed'] = df['Allowed'].fillna(True)  # The first login should be allowed
    return df

limited_logins = merged_data.groupby('Aggregator').apply(limit_logins)
allowed_logins_count = limited_logins[limited_logins['Allowed']].groupby('Aggregator').size()
print("Allowed logins count per aggregator under the new policy:\n", allowed_logins_count)

impact = (volume_per_aggregator - allowed_logins_count) / volume_per_aggregator * 100
print("Impact of the new policy (reduction in allowed logins) per aggregator (%):\n", impact)


In [17]:
login_transactions

,IP,LoginSuccess,AccountName,timestamp
0,54.208.59.10,Fail,756bb790d96873a,2018-10-28 05:00:00.016
1,206.108.41.103,Success,7b3eb9367400c1b,2018-10-28 05:00:00.022
2,216.34.61.115,Success,abfa625be56908a,2018-10-28 05:00:00.041
3,52.44.118.176,Fail,dad678e942a965c,2018-10-28 05:00:00.046
4,64.41.181.29,Success,e3014e35d5ec49b,2018-10-28 05:00:00.067
...,...,...,...,...
4092248,216.34.61.225,Fail,8dd393aa2fc2d3e,2018-10-29 04:59:59.913
4092249,216.34.61.167,Success,ac59730a398c7f0,2018-10-29 04:59:59.929
4092250,54.208.59.10,Fail,33f59293e9e34eb,2018-10-29 04:59:59.939
4092251,54.208.59.10,Fail,e0e0836a6e4833c,2018-10-29 04:59:59.961


In [97]:
import pandas as pd
import numpy as np
import matplotlib as plt
print('lets start preprocessing data')
# Load the data
aggregator_ip_list = pd.read_csv(r'C:\Users\ARMANDO\Documents\Python Scripts\Challenges\F5\Aggregator IP List.csv', index_col=0)
login_transactions = pd.read_csv(r'C:\Users\ARMANDO\Documents\Python Scripts\Challenges\F5\Login Transactions.csv', index_col=0)

print(login_transactions.dtypes)
print('----------------------------------')
print(aggregator_ip_list.dtypes)

# lets see the overal info from files

print(aggregator_ip_list.describe().round(2))
print('-------------------------------------------------')
print(login_transactions.describe().round(2))
#we can see IP \neq ip, so lets change to IP instead of ip

print('done!')

print('***********************next process*************************')
aggregator_ip_list.rename(columns={'ip': 'IP'}, inplace=True)
# aggregator_ip_list.head()

# Convert the timestamp to a datetime object for easier manipulation
login_transactions['timestamp'] = pd.to_datetime(login_transactions['timestamp'], unit='ms')
print('LoginSuccess is not numeric, converting to boolean (0, 1)')
login_transactions['LoginSuccess'] = login_transactions['LoginSuccess'].map({'Fail': 0, 'Success': 1})

print('format complete!')
print('generating master table...')
# Merge the dataframes to add aggregator names to the login transactions
merged_data = login_transactions.merge(aggregator_ip_list, on='IP', how='inner')
merged_data.dropna(subset=['Aggregator'])
print(merged_data.head(1))
print(merged_data.describe().round(2))
print('------------------------------------------------------------------------')
print(merged_data.head(5))

lets start preprocessing data
IP              object
LoginSuccess    object
AccountName     object
timestamp        int64
dtype: object
----------------------------------
ip            object
Aggregator    object
dtype: object
                  ip Aggregator
count            315        315
unique           315          7
top     54.208.59.10  YoungOnes
freq               1        249
-------------------------------------------------
          timestamp
count  4.092253e+06
mean   1.540746e+12
std    2.496030e+07
min    1.540703e+12
25%    1.540724e+12
50%    1.540748e+12
75%    1.540768e+12
max    1.540789e+12
done!
***********************next process*************************
LoginSuccess is not numeric, converting to boolean (0, 1)
format complete!
generating master table...
             IP  LoginSuccess      AccountName               timestamp  \
0  54.208.59.10             0  756bb790d96873a 2018-10-28 05:00:00.016   

  Aggregator  
0        AWS  
       LoginSuccess                

In [98]:
# 1. Which aggregators are accessing Banc ABC’s systems?
aggregators_accessing = merged_data['Aggregator'].unique()
print(aggregators_accessing)
print('------------------------------------------------------')
# 2. How much volume are the aggregators sending?
aggregator_volume = merged_data['Aggregator'].value_counts()
print(aggregator_volume)

['AWS' 'Insight' 'YoungOnes' 'FinTech' 'PayTM' 'FunTown']
------------------------------------------------------
Aggregator
AWS          2182274
YoungOnes     918552
FinTech       309554
Insight       213934
PayTM          90280
FunTown            8
Name: count, dtype: int64


In [99]:
# 3. How many individual user accounts are being accessed by the aggregators?
individual_accounts = merged_data.groupby('Aggregator')['AccountName'].nunique()
print(individual_accounts)
print('-------------------------------------------------------------------------')
aggregators = merged_data['Aggregator'].unique()
print(aggregators)


Aggregator
AWS          1923202
FinTech       204821
FunTown            5
Insight       181749
PayTM          83403
YoungOnes     479764
Name: AccountName, dtype: int64
-------------------------------------------------------------------------
['AWS' 'Insight' 'YoungOnes' 'FinTech' 'PayTM' 'FunTown']


In [100]:
#4. What is the login success rate of these aggregators and is this in line with what would be expected in your opinion?
print(merged_data['LoginSuccess'].unique())
print('success rate:...')
success_rate_per_aggregator = merged_data.groupby('Aggregator')['LoginSuccess'].mean().round(2)
print("Login success rate per aggregator:\n", success_rate_per_aggregator)

[0 1]
success rate:...
Login success rate per aggregator:
 Aggregator
AWS          0.00
FinTech      0.00
FunTown      0.38
Insight      0.95
PayTM        0.15
YoungOnes    0.70
Name: LoginSuccess, dtype: float64


In [102]:
merged_data['Timestamp']=merged_data['timestamp']
merged_data

,IP,LoginSuccess,AccountName,timestamp,Aggregator,Timestamp
0,54.208.59.10,0,756bb790d96873a,2018-10-28 05:00:00.016,AWS,2018-10-28 05:00:00.016
1,206.108.41.103,1,7b3eb9367400c1b,2018-10-28 05:00:00.022,Insight,2018-10-28 05:00:00.022
2,216.34.61.115,1,abfa625be56908a,2018-10-28 05:00:00.041,YoungOnes,2018-10-28 05:00:00.041
3,52.44.118.176,0,dad678e942a965c,2018-10-28 05:00:00.046,FinTech,2018-10-28 05:00:00.046
4,64.41.181.29,1,e3014e35d5ec49b,2018-10-28 05:00:00.067,YoungOnes,2018-10-28 05:00:00.067
...,...,...,...,...,...,...
3714597,216.34.61.225,0,8dd393aa2fc2d3e,2018-10-29 04:59:59.913,YoungOnes,2018-10-29 04:59:59.913
3714598,216.34.61.167,1,ac59730a398c7f0,2018-10-29 04:59:59.929,YoungOnes,2018-10-29 04:59:59.929
3714599,54.208.59.10,0,33f59293e9e34eb,2018-10-29 04:59:59.939,AWS,2018-10-29 04:59:59.939
3714600,54.208.59.10,0,e0e0836a6e4833c,2018-10-29 04:59:59.961,AWS,2018-10-29 04:59:59.961


In [103]:
# 5.- What is the average number of transactions each aggregator sends per 10min interval?

# merged_data.reset_index()
merged_data.set_index('timestamp', inplace=True) # ste set index es problematico
average_transactions_per_10min = merged_data.groupby('Aggregator').resample('10T').size().groupby('Aggregator').mean()
print("Average number of transactions per 10-minute interval per aggregator:\n", round(average_transactions_per_10min, 2))


# merged_data['timestamp'] = merged_data['timestamp'].floor('10T')
# average_transactions_per_10min = merged_data.groupby(['Aggregator', 'TimeInterval']).size().groupby('Aggregator').mean()


C:\Users\ARMANDO\AppData\Local\Temp\ipykernel_13456\382329385.py:5: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  average_transactions_per_10min = merged_data.groupby('Aggregator').resample('10T').size().groupby('Aggregator').mean()


Average number of transactions per 10-minute interval per aggregator:
 Aggregator
AWS          15154.68
FinTech       2149.68
FunTown          0.53
Insight       1485.65
PayTM          626.94
YoungOnes     6378.83
dtype: float64


In [104]:
merged_data2=merged_data.reset_index()
merged_data2

,timestamp,IP,LoginSuccess,AccountName,Aggregator,Timestamp
0,2018-10-28 05:00:00.016,54.208.59.10,0,756bb790d96873a,AWS,2018-10-28 05:00:00.016
1,2018-10-28 05:00:00.022,206.108.41.103,1,7b3eb9367400c1b,Insight,2018-10-28 05:00:00.022
2,2018-10-28 05:00:00.041,216.34.61.115,1,abfa625be56908a,YoungOnes,2018-10-28 05:00:00.041
3,2018-10-28 05:00:00.046,52.44.118.176,0,dad678e942a965c,FinTech,2018-10-28 05:00:00.046
4,2018-10-28 05:00:00.067,64.41.181.29,1,e3014e35d5ec49b,YoungOnes,2018-10-28 05:00:00.067
...,...,...,...,...,...,...
3714597,2018-10-29 04:59:59.913,216.34.61.225,0,8dd393aa2fc2d3e,YoungOnes,2018-10-29 04:59:59.913
3714598,2018-10-29 04:59:59.929,216.34.61.167,1,ac59730a398c7f0,YoungOnes,2018-10-29 04:59:59.929
3714599,2018-10-29 04:59:59.939,54.208.59.10,0,33f59293e9e34eb,AWS,2018-10-29 04:59:59.939
3714600,2018-10-29 04:59:59.961,54.208.59.10,0,e0e0836a6e4833c,AWS,2018-10-29 04:59:59.961


In [106]:
# 6. What is the maximum number of transactions each aggregator sends per 10min interval?
max_transactions_per_10min = merged_data.groupby('Aggregator').resample('10T').size().groupby('Aggregator').max()
print("Maximum number of transactions per 10-minute interval per aggregator:\n", max_transactions_per_10min)


C:\Users\ARMANDO\AppData\Local\Temp\ipykernel_13456\3605265967.py:2: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  max_transactions_per_10min = merged_data.groupby('Aggregator').resample('10T').size().groupby('Aggregator').max()


Maximum number of transactions per 10-minute interval per aggregator:
 Aggregator
AWS          16238
FinTech       6903
FunTown          3
Insight       5960
PayTM          744
YoungOnes     9535
dtype: int64


In [110]:

#7
def limit_logins(df):
    df = df.sort_values(by='Timestamp')
    df['Allowed'] = df.groupby(['AccountName', 'Aggregator'])['Timestamp'].diff().gt(pd.Timedelta(minutes=10))
    df['Allowed'] = df['Allowed'].fillna(True)  # The first login should be allowed
    return df

limited_logins = merged_data.groupby('Aggregator').apply(limit_logins).reset_index(drop=True)
allowed_logins_count = limited_logins[limited_logins['Allowed']].groupby('Aggregator').size()
print("Allowed logins count per aggregator under the new policy:\n", allowed_logins_count)

impact = (aggregator_volume - allowed_logins_count) / aggregator_volume * 100
print("Impact of the new policy (reduction in allowed logins) per aggregator (%):\n", impact)

C:\Users\ARMANDO\AppData\Local\Temp\ipykernel_13456\2748726671.py:7: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  limited_logins = merged_data.groupby('Aggregator').apply(limit_logins).reset_index(drop=True)


Allowed logins count per aggregator under the new policy:
 Aggregator
AWS          161937
FinTech       96701
FunTown           2
Insight       17726
PayTM          2026
YoungOnes    157985
dtype: int64
Impact of the new policy (reduction in allowed logins) per aggregator (%):
 Aggregator
AWS          92.579438
FinTech      68.761185
FunTown      75.000000
Insight      91.714267
PayTM        97.755871
YoungOnes    82.800647
dtype: float64


In [111]:


# Count the number of times this exceeds 1 (i.e., the excess transactions)
excess_transactions = transactions_per_account_10min[transactions_per_account_10min > 1] - 1
impact_of_limitation = excess_transactions.groupby('Aggregator').sum()

# Prepare the results for presentation
results = {
    'Aggregators Accessing': aggregators_accessing,
    'Aggregator Volume': aggregator_volume,
    'Individual Accounts': individual_accounts,
    'Login Success Rate': login_success_rate,
    'Average Transactions per 10min': average_transactions_per_10min,
    'Max Transactions per 10min': max_transactions_per_10min,
    'Impact of Limitation': impact_of_limitation
}

# Display results
for key, value in results.items():
    print(f"{key}:\n{value}\n")



NameError: name 'transactions_per_account_10min' is not defined

In [18]:
#Vamos a abrir los archivos para visualizarlos solamente:
agg.head()

,ip,Aggregator
0,54.208.59.10,AWS
1,54.88.74.128,AWS
2,54.88.202.28,AWS
3,52.70.160.54,FinTech
8,52.44.118.176,FinTech


In [15]:
import pandas as pd
tr=pd.read_csv(r'C:\Users\ARMANDO\Documents\Python Scripts\Challenges\F5\Login Transactions.csv', index_col=0 )
agg=pd.read_csv(r'C:\Users\ARMANDO\Documents\Python Scripts\Challenges\F5\Aggregator IP List.csv', index_col=0)
tr

,IP,LoginSuccess,AccountName,timestamp
0,54.208.59.10,Fail,756bb790d96873a,1540702800016
1,206.108.41.103,Success,7b3eb9367400c1b,1540702800022
2,216.34.61.115,Success,abfa625be56908a,1540702800041
3,52.44.118.176,Fail,dad678e942a965c,1540702800046
4,64.41.181.29,Success,e3014e35d5ec49b,1540702800067
...,...,...,...,...
4092248,216.34.61.225,Fail,8dd393aa2fc2d3e,1540789199913
4092249,216.34.61.167,Success,ac59730a398c7f0,1540789199929
4092250,54.208.59.10,Fail,33f59293e9e34eb,1540789199939
4092251,54.208.59.10,Fail,e0e0836a6e4833c,1540789199961


In [19]:
agg['Aggregator'].unique()

array(['AWS', 'FinTech', 'Insight', 'YoungOnes', 'PayTM', 'Funtown',
       'FunTown'], dtype=object)

In [20]:
agg['ip'].unique()

array(['54.208.59.10', '54.88.74.128', '54.88.202.28', '52.70.160.54',
       '52.44.118.176', '52.70.161.90', '52.0.123.48', '206.108.41.101',
       '206.108.41.102', '206.108.41.103', '34.230.198.253',
       '34.228.103.5', '34.230.198.27', '34.232.227.45', '206.108.41.105',
       '206.108.41.106', '206.108.41.107', '206.108.41.108',
       '64.41.181.183', '64.41.181.155', '64.41.181.105', '216.34.61.75',
       '216.34.61.165', '216.34.61.191', '216.34.61.157', '216.34.61.181',
       '64.41.181.177', '64.41.181.157', '216.34.61.115', '216.34.61.89',
       '64.41.181.215', '64.41.181.135', '216.34.61.155', '64.41.181.17',
       '216.34.61.183', '216.34.61.213', '216.34.61.133', '216.34.61.199',
       '216.34.61.99', '64.41.181.149', '216.34.61.167', '216.34.61.93',
       '216.34.61.193', '64.41.181.127', '216.34.61.109', '216.34.61.103',
       '64.41.181.93', '64.41.181.185', '216.34.61.229', '216.34.61.189',
       '64.41.181.43', '216.34.61.53', '64.41.181.97', '216.34.61